In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('Tidy_Cells_DDM_Normalized_Proteins.tsv', delimiter='\t')

### Remove Contaminants

In [4]:
df[df['Contaminant'] == True]['Accession'].nunique()  # this is the number of proteins that are contaminants

13

In [5]:
df = df[df['Contaminant'] == False]  # remove the contaminants

### Drop Medium Confidence Proteins

In [6]:
df[df['Protein FDR Confidence: Combined'] == 'Medium']['Accession'].nunique()  # this is the number of proteins with medium confidence

203

In [7]:
df = df[df['Protein FDR Confidence: Combined'] == 'High'] # keep only the proteins with high confidence

In [8]:
df['Accession'].nunique() # the number of proteins left after filtering

1605

### Keep Relevant Columns

In [9]:
df = df[['Accession', 'Sample', 'Abundances (Normalized):']]

In [10]:
df = df.pivot(index='Sample', columns='Accession', values='Abundances (Normalized):')

In [11]:
df['Sample'] = df.index
df = df.reset_index(drop=True)

In [13]:
df_batches = pd.read_csv('Cells_DDM_Normalized_InputFiles_Reordered.tsv', delimiter='\t')

In [14]:
df_batches['Chip'] = df_batches['File Name'].str.extract(r'(_[a-zA-Z]*_)')
df_batches['Chip'].value_counts()

_QCChip_     83
_RedChip_    77
_QCHeLa_      9
Name: Chip, dtype: int64

In [15]:
df_batches['Creation Date'] = pd.to_datetime(df_batches['Creation Date'])
df_batches['Creation Date'] = pd.DatetimeIndex(df_batches['Creation Date']).day
df_batches.groupby('Chip')[['Chip','Creation Date']].agg(['unique'])

,Chip,Creation Date
,unique,unique
Chip,,
_QCChip_,[_QCChip_],"[13, 14]"
_QCHeLa_,[_QCHeLa_],[6]
_RedChip_,[_RedChip_],"[14, 15]"


In [16]:
df_batches = df_batches[['File ID', 'Chip', 'Creation Date', 'Side']]

In [17]:
df_combined = pd.merge(df, df_batches, left_on='Sample', right_on='File ID', how='left')
df = df_combined

### Remove proteins that aren't in enough cells

In [18]:
proteins = list(df.drop(['Sample', 'File ID', 'Chip', 'Creation Date', 'Side'], axis=1).columns)

In [23]:
n_cells = df[proteins].index.shape[0]
perc_cells_w_protein = (n_cells - df[proteins].isna().sum()) / n_cells
perc_cells_w_protein[perc_cells_w_protein < 0.5].shape  # the number of proteins that are in less than 50% of the cells

(988,)

In [24]:
proteins_to_drop = perc_cells_w_protein[perc_cells_w_protein < 0.5].index

In [25]:
df = df.drop(proteins_to_drop, axis=1)

### Remove cells without enough proteins

In [26]:
proteins = list(df.drop(['Sample', 'File ID', 'Chip', 'Creation Date', 'Side'], axis=1).columns)

In [27]:
df.index = df['Sample']

In [33]:
n_proteins = df[proteins].transpose().index.shape[0]
perc_proteins_in_cells = (n_proteins - df[proteins].transpose().isna().sum()) / n_proteins
cells_to_drop = perc_proteins_in_cells[perc_proteins_in_cells < 0.5].index
cells_to_drop

Index(['F1', 'F131', 'F14'], dtype='object', name='Sample')

In [34]:
df = df.transpose().drop(cells_to_drop, axis=1)

In [35]:
df = df.transpose()

In [36]:
df

,B5ME19,O00231,O00299,O00422,O00571,O00764,O14745,O14818,O15355,O15372,...,Q9Y3U8,Q9Y490,Q9Y4L1,Q9Y5B9,Q9Y617,Sample,File ID,Chip,Creation Date,Side
Sample,,,,,,,,,,,,,,,,,,,,,
F10,3373690.27517,2991493.2171,1576766.184773,4786781.858305,10751392.024809,2687357.162864,1397848.460283,NaN,NaN,1004919.866468,...,6191537.311871,10334789.410666,201631429.799781,11285654.752135,1323828.196699,F10,F10,_QCChip_,14.0,Left
F100,2299355.790048,2241388.630235,NaN,NaN,16039892.331403,NaN,2025436.410945,1501850.376526,51378934.023496,957970.594455,...,2808157.05697,17164232.680574,78660913.994576,NaN,4925942.355848,F100,F100,_RedChip_,15.0,Right
F101,3113306.648528,1809834.035484,6120705.505014,5670456.799759,5583600.947291,3016982.482661,2792555.733788,2168849.683538,NaN,NaN,...,2877289.038176,6974637.699785,288871882.301377,12436373.747393,4420583.371715,F101,NaN,NaN,NaN,NaN
F102,1856249.798234,1944483.03957,4652592.705573,5736307.433317,9676142.786788,3968779.323402,6284356.472783,2174116.550347,NaN,901840.37001,...,5601243.514291,11443082.755791,245512837.390592,14401675.520609,5427335.083864,F102,F102,_RedChip_,15.0,Left
F103,2175885.871595,2219012.907581,3233294.323624,3313285.295724,10289915.281581,NaN,1092045.478096,858167.908599,42892793.438634,1344367.999387,...,3665115.377148,14310882.831074,165779607.52715,9431326.715832,2655200.200714,F103,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
F95,2150390.115804,1598873.50015,1758013.520807,656003.634326,15650965.126155,NaN,NaN,NaN,9591594.107522,991872.837533,...,5003449.498178,26530411.441392,NaN,NaN,NaN,F95,F95,_QCChip_,13.0,Right
F96,2835923.042279,1045652.777073,NaN,5428826.513596,8817327.012357,NaN,NaN,NaN,NaN,770715.672522,...,4037650.764888,13400416.107579,NaN,8484228.528682,2025257.360082,F96,F96,_QCChip_,13.0,Left
F97,2715682.633229,2016341.300724,1544609.600587,987079.080646,8698076.336869,1236266.020425,1632730.40379,NaN,3785285.870863,659903.1831,...,4982011.745528,23126100.340803,NaN,NaN,NaN,F97,F97,_QCChip_,13.0,Right


In [38]:
df.to_csv('cleaned_filtered_data.csv', index=False)